In [2]:
import re
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#!pip install seaborn
#import seaborn as sns
#from datetime import *
#import datetime as dt
#from matplotlib.ticker import MaxNLocator
#!pip install regex
import regex
#!pip install emoji
#import emoji
#from seaborn import *
#!pip install heatmap
#from heatmap import heatmap
#!pip install wordcloud
#from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
#from nltk import *
#from plotly import express as px

### Python code to extract Date from chat file 

In [3]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    #pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}' # for New Group settings
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [4]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [5]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [6]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'WhatsApp_Chat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    
    #print(fp.readline())
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

29957


In [7]:
df

,Date,Time,Author,Message
0,2021-05-11,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...
1,2021-05-11,8:33 PM,Celal PUCIT,Karain gey 👍
2,2021-05-11,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,2021-05-11,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,2021-05-11,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶
...,...,...,...,...
29952,2021-08-17,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...
29953,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr
29954,2021-08-17,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂
29955,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂


In [8]:
### Checking shape of dataset.
df.shape

(29957, 4)

In [9]:
### Checking basic information of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29957 entries, 0 to 29956
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     29957 non-null  datetime64[ns]
 1   Time     29957 non-null  object        
 2   Author   29957 non-null  object        
 3   Message  29957 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 936.3+ KB


In [10]:
### Checking no. of null values in dataset
df.isnull().sum()

### Checking head part of dataset
#df.head(50)

### Checking tail part of dataset
#df.tail(50)

Date       0
Time       0
Author     0
Message    0
dtype: int64

In [11]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df = df.replace(r'<Media omitted>$', np.nan , regex=True)
df.shape

df = df[df['Message'].notna()]

In [12]:
### Checking no. of authors of group
df['Author'].nunique()

34

In [13]:
### Checking authors of group
#df['Author'].unique()

In [14]:
df.isna()
#df['Message']

,Date,Time,Author,Message
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
29952,False,False,False,False
29953,False,False,False,False
29954,False,False,False,False
29955,False,False,False,False


In [15]:
df.shape

(27694, 4)

In [16]:
### Checking authors of group
df['Author'].unique()

array(['Ayesha PUCIT', 'Celal PUCIT', 'F.Bilal Uol', 'Umar Ghaffar PUCIT',
       'Amna Ali PUCIT', 'Ayesha Kiran PUCIT', 'Abdullah PUCIT',
       '+92 323 4446586', 'Khurram PUCIT', '+92 322 4155225',
       'Saad PUCIT', 'Adnan Ashraf PUCIT', 'Iqra Aslam PUCIT',
       'Shakir Jamil', 'Muflah PUCIT', 'Arshia CH PUCIT',
       'Usman CH PUCIT', '+92 304 5307578', '+92 300 4611646',
       'Muhammad Faraz PUCIT', '+92 306 2628299', 'Hafsa PUCIT',
       '+92 321 7222267', '+92 307 3900838', 'Aimen PUCIT',
       '+92 303 5018620', 'Nabeel Jutt', '+92 322 6678540',
       'Saad PUCIT MSV', '+92 335 0607000', '+92 335 5027377',
       '+92 306 4093886', 'Aneesa PUCIT', '+92 303 9326488'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [17]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)
### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
0,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...,57,12,0,0
1,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Karain gey 👍,12,3,0,0
2,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta...",51,7,0,0
3,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,46,10,0,0
4,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶,46,11,0,0
...,...,...,...,...,...,...,...,...,...
29952,2021-08-17,Tuesday,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...,77,17,0,0
29953,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr,20,5,0,0
29954,2021-08-17,Tuesday,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂,27,6,0,0
29955,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂,3,1,0,0


In [18]:
### Extracting basic statistics from the dataset
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

Group Chatting Stats : 
Total Number of Messages : 27694
Total Number of Media Messages : 0
Total Number of Links : 0


In [19]:
#Extracting basic statistics of each user : 
l = df.Author.unique()
for i in range(len(l)):
  ### Filtering out messages of particular user
  req_df = df[df["Author"] == l[i]]
  ### req_df will contain messages of only one particular user
  print(f'--> Stats of {l[i]} <-- ')
  ### shape will print number of rows which indirectly means the number of messages
  print('Total Message Sent : ', req_df.shape[0])
  ### Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word\'s']))/req_df.shape[0]
  w_p_m = ("%.3f" % round(words_per_message, 2))  
  print('Average Words per Message : ', w_p_m)
  ### media conists of media messages
  media = sum(req_df["Media_Count"])
  print('Total Media Message Sent : ', media)
  ### links consist of total links
  links = sum(req_df["Url_Count"])   
  print('Total Links Sent : ', links)   
  print()
  print('----------------------------------------------------------n')

--> Stats of Ayesha PUCIT <-- 
Total Message Sent :  1619
Average Words per Message :  5.920
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Celal PUCIT <-- 
Total Message Sent :  617
Average Words per Message :  7.170
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of F.Bilal Uol <-- 
Total Message Sent :  3338
Average Words per Message :  9.190
Total Media Message Sent :  77
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Umar Ghaffar PUCIT <-- 
Total Message Sent :  1649
Average Words per Message :  7.010
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Amna Ali PUCIT <-- 
Total Message Sent :  230
Average Words per Message :  8.260
Total Media Message Sent :  0
Total Links Sent :  0

-------------------------

In [20]:
#conversationPath = 'WhatsApp_Chat.txt' # chat file
conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    line = fp.readline()
    line = fp.readline()
    line = fp.readline()
    line = fp.readline()
    

    
print(line)

conversationPath = 'WhatsApp_Chat.txt' # chat file
#conversationPath = 'GroupChat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    line2 = fp.readline()
    line2 = fp.readline()
    line2 = fp.readline()
    line2 = fp.readline()
    

    
print(line2)

11/08/2021, 7:21 pm - HR: Why you created this Group😄

5/11/21, 8:33 PM - Ayesha PUCIT: Kbhe sessional, kbhe assignment  kbhe presentations



In [21]:
date, time, author, message = getDataPoint(line2)
print(date)

print(type(line))
print(type(line2))

pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}'
result = re.match(pattern, line2)
print(result)

if startsWithDateAndTime(line2): 
    #print(100)
    if len(messageBuffer) > 0:
        #print('!!!!!!!!!!!!!!!!!')
        parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
    messageBuffer.clear() 
    date, time, author, message = getDataPoint(line) 
    messageBuffer.append(message)
else:
    print('NO')

5/11/21
<class 'str'>
<class 'str'>
<re.Match object; span=(0, 7), match='5/11/21'>


In [42]:
#data = [df1["A"], df2["A"]]
#headers = ["df1", "df2"]
#df3 = pd. concat(data, axis=1, keys=headers)

data = [df['Message']]
rd = pd.concat(data, axis=1)

type(rd)

pandas.core.frame.DataFrame

In [43]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27694 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  27694 non-null  object
dtypes: object(1)
memory usage: 432.7+ KB


In [44]:
### new Data Frame
#rd = df['Message']

### Remove all Emoji and other Symbols
rd = rd.replace(r'\W*$', '' , regex=True)

rd = rd.replace(r'^\s*$', np.nan , regex=True)

#rd = df[df['Message'].notna()]
rd = rd[rd['Message'].notna()]

In [45]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24457 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  24457 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [46]:
rd

,Message
0,Hmme na pagal ho chuki har teacher ki mintaien...
1,Karain gey
2,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,Or phir teachers k krny waly kam b me he kron
...,...
29951,😂😂😂😂 Bus Focus on your study you will get soon
29952,I don't need that type of aurat jo MSCS ki wja...
29953,😂😂😂 Fir to chutti kr
29954,Aho chutti hi ki hoi hai


In [47]:
rd = rd.sample(frac = 1)

In [48]:
len(rd)

mode = [''] * len(rd)

print(mode[:10])

rd['Mode'] = mode

### Rearranging the columns for better understanding
rd = rd[['Mode','Message']]

rd['Words'] = rd['Message'].apply(lambda s : len(s.split(' ')))

['', '', '', '', '', '', '', '', '', '']


In [49]:
rd

,Mode,Message,Words
10061,,Hn g,2
5690,,Han i agree,3
29702,,tu dkh idr kya leny aya,6
13150,,Han lein gy,3
1496,,"🙉🙉🙉🙉 Q? Umar bilal, shakir in ko nai soch skty",10
...,...,...,...
8777,,Yarrr ignore kru please,4
11918,,Parhae se jaan jate h ab tu,7
18567,,Aj to ap aai thi,5
29188,,Ohh ik to ya dukh,5


In [58]:
#rd.to_csv('data.csv', index= False)  ## file size 20k+ ......

In [56]:
rd = rd[rd.Words != 1]
rd = rd[rd.Words != 2]

In [57]:
rd

,Mode,Message,Words
5690,,Han i agree,3
29702,,tu dkh idr kya leny aya,6
13150,,Han lein gy,3
1496,,"🙉🙉🙉🙉 Q? Umar bilal, shakir in ko nai soch skty",10
4335,,Aby chup kr k send kr. Q sunna chatha ha subha...,12
...,...,...,...
2577,,Bchy ny bt e aisi ki hai ... Ku @923054524120,10
8777,,Yarrr ignore kru please,4
11918,,Parhae se jaan jate h ab tu,7
18567,,Aj to ap aai thi,5


In [62]:
import emoji


def char_is_emoji(character):
    return character in emoji.UNICODE_EMOJI


def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False


if __name__ == '__main__':
    print(char_is_emoji(u'\u2764'))
    print(text_has_emoji('🙉🙉🙉🙉 Q? Umar bilal, shakir in ko nai soch skty'))

False
False
